In [1]:
%run preprocessing.py

import data
merge orders and order_products__prior
features extraction...
size of prd features : 9
size of users features : 18
nb of usersXproducts features : 12
merge prod, user and usersXprod feature on usersXprod
merge train reordered to usersXprod
split train and test
size of training set : 8474661
size of testing set : 4833292
Size of features of dataset : 36
Class weight of 0 : 0.9021997458069414
Class weight of 1 : 0.09780025419305857


In [2]:
train_1 = train[(train.reordered == 1)]
nb_ = train_1.shape[0]
train_0 = train[(train.reordered == 0)]
train_0 = train_0.sample(n = nb_)
print(train_1.shape)
print(train_0.shape)

(828824, 37)
(828824, 37)


In [4]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier, VotingClassifier


/Users/Shawn/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
train_new = pd.concat([train_0, train_1])
X = train_new.drop(["reordered"], axis = 1)
y = train_new[["reordered"]]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)


In [6]:
def validation(model):
    model.fit(X_train, y_train)
    print(str(model), "cross_val down")
    y_pred = model.predict(X_val).reshape(y_val.shape[0], 1)
    print('Misclassified samples: %d' % (y_val != y_pred).sum())
    print('Accuracy : %.2f' % ((y_val == y_pred).sum() / y_val.shape[0]))
    print('Accuracy (sklearn): %.2f' % accuracy_score(y_val, y_pred))
    print("=== Confusion Matrix ===")
    print(confusion_matrix(y_val, y_pred, labels = [0,1], sample_weight = None))
    print("=== F1-score ===")
    print(f1_score(y_val, y_pred, labels=[0,1]))

### 整理過的testing dataset 去預測
### 拿出以前的商品顧客組合去預測是否reordered    
### 建立一個門檻值去篩選是否購買???
### 選出指定order_id 購買的商品並匯出
def cross_val_and_testing(df, df_test, model, threshold):
    model.fit(X, y)
#    print (cross_val_score(model, np.array(X), np.array(y).reshape(y.shape[0], ), cv=5))
    print ("Fitting Over")
    pred = pd.DataFrame(model.predict_proba(df_test), columns = ["prob_0", "prob_1"])
    df["prediction"] = pred["prob_1"]
    df = df[["order_id", "product_id", "prediction"]]
    print (df.head())

    con = (df.prediction > threshold)
    test0_0 = df[~con]
    test0_1 = df[con]
    print ("Percent of test0 is 0 :", test0_0.shape[0])#/df.shape[0])
    print ("Percent of test0 is 1 :", test0_1.shape[0])#/df.shape[0])

    f = lambda x: np.array_split(list(set(x)), 1)[0]
    order_prod = test0_1.groupby('order_id')["product_id"].apply(f).reset_index()
    order_id = df.groupby("order_id")["product_id"].apply(f).reset_index()
    submit = pd.merge(order_id[["order_id"]], order_prod, how='left', on=['order_id'])
    return submit

In [7]:
lg = LogisticRegression(penalty = 'l2',
                        dual = False,
                        tol = 0.0001,
                        C = 0.01,
                        fit_intercept = True,
                        intercept_scaling = 1,
                        random_state = 7,
                        solver = 'liblinear',
                        max_iter = 1000,
                        multi_class = 'ovr',
                        verbose = 0,
                        warm_start = False,
                        n_jobs = 1)


In [8]:
threshold = 0.05

validation(lg)
submit = cross_val_and_testing(df=test0, df_test=test, model=lg, threshold=threshold)
submit.to_csv("submit_lg.csv", index = False)

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=7, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False) cross_val down
Misclassified samples: 85864
Accuracy : 0.74
Accuracy (sklearn): 0.74
=== Confusion Matrix ===
[[121853  44133]
 [ 41731 123813]]
=== F1-score ===
0.742529011365
Fitting Over
     order_id  product_id  prediction
120   2774568         248      0.3369
121   2774568        1005      0.5888
122   2774568        1819      0.5799
123   2774568        7503      0.5020
124   2774568        8021      0.6054
Percent of test0 is 0 : 3243187
Percent of test0 is 1 : 1590105


In [9]:
df = test0.copy()
df_test = test
model = lg
threshold = 0.05

model.fit(X, y)
print ("Fitting Over")
pred = pd.DataFrame(model.predict_proba(df_test), columns = ["prob_0", "prob_1"])
df["prediction"] = pred["prob_1"]
df = df[["order_id", "product_id", "prediction"]]
print (df.head())

con = (df.prediction > threshold)
test0_0 = df[~con]
test0_1 = df[con]
print ("Percent of test0 is 0 :", test0_0.shape[0])#/df.shape[0])
print ("Percent of test0 is 1 :", test0_1.shape[0])#/df.shape[0])

f = lambda x: np.array_split(list(set(x)), 1)[0]
order_prod = test0_1.groupby('order_id')["product_id"].apply(f).reset_index()
order_id = df.groupby("order_id")["product_id"].apply(f).reset_index()
submit = pd.merge(order_id[["order_id"]], order_prod, how='left', on=['order_id'])


Fitting Over
     order_id  product_id  prediction
120   2774568         248      0.3369
121   2774568        1005      0.5888
122   2774568        1819      0.5799
123   2774568        7503      0.5020
124   2774568        8021      0.6054
Percent of test0 is 0 : 3243187
Percent of test0 is 1 : 1590105


In [10]:
submit.isnull().sum()

order_id          0
product_id    47751
dtype: int64

In [12]:
len(set(test0_1.order_id)), len(test0_1.order_id)

(27249, 1590105)

In [16]:
test0_1[(test0_1.order_id == 1735923)].shape[0]

117

In [15]:
test0_0[(test0_0.order_id == 1735923)].shape[0]

16